In [17]:
import numpy as np
import pandas as pd
import json

In [18]:
ALL_STATES= ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']#, 'PR']

ACC_STATES = ['CA', 'TX', 'FL', 'OR', 'MN', 'VA', 'SC', 'NY', 'PA', 'NC', 'TN', 'MI', 'MO']

In [19]:
root_dir = '../results/'
task = 'income'
embedding_method_list = ['one_hot', 'table_to_text']
source = 'CA'
model_name = 'xgb'
experiment_id_list = [i for i in range(150)]
metric_list = ["test_result_acc", "test_result_f1"]

# Initialize an empty list to collect data
data_records = []

for embedding_method in embedding_method_list:
    prefix = f'{root_dir}/{task}/{embedding_method}/{source}/{model_name}/'
    # Loop through experiment IDs
    for experiment_id in experiment_id_list:
        path = prefix + f'{experiment_id}.json'
        print(path)
        with open(path, 'r') as file:
            # Load its content and convert it into a Python dictionary
            data = json.load(file)
            # Extract required metrics and store in the list
            for metric in metric_list:
                for state in ALL_STATES:
                    value = data.get(metric, {}).get(state, None)  # Nested get to handle missing data gracefully
                    record = {
                        'experiment_id': experiment_id,
                        'metric': metric,
                        'state': state,
                        'embedding_method': embedding_method,
                        'value': value
                    }
                    data_records.append(record)
# Convert the list of records into a DataFrame
df = pd.DataFrame(data_records)

../results//income/one_hot/CA/xgb/0.json
../results//income/one_hot/CA/xgb/1.json
../results//income/one_hot/CA/xgb/2.json
../results//income/one_hot/CA/xgb/3.json
../results//income/one_hot/CA/xgb/4.json
../results//income/one_hot/CA/xgb/5.json
../results//income/one_hot/CA/xgb/6.json
../results//income/one_hot/CA/xgb/7.json
../results//income/one_hot/CA/xgb/8.json
../results//income/one_hot/CA/xgb/9.json
../results//income/one_hot/CA/xgb/10.json
../results//income/one_hot/CA/xgb/11.json
../results//income/one_hot/CA/xgb/12.json
../results//income/one_hot/CA/xgb/13.json
../results//income/one_hot/CA/xgb/14.json
../results//income/one_hot/CA/xgb/15.json
../results//income/one_hot/CA/xgb/16.json
../results//income/one_hot/CA/xgb/17.json
../results//income/one_hot/CA/xgb/18.json
../results//income/one_hot/CA/xgb/19.json
../results//income/one_hot/CA/xgb/20.json
../results//income/one_hot/CA/xgb/21.json
../results//income/one_hot/CA/xgb/22.json
../results//income/one_hot/CA/xgb/23.json
..

In [20]:
# Group the DataFrame and apply multiple aggregation functions to the 'value' column
grouped_df = df.groupby(['metric', 'state', 'embedding_method', ])['value'].agg(['mean', 'max', 'min', 'std']).reset_index()

# Rename the columns for clarity (optional)
grouped_df.columns = ['metric', 'state', 'embedding_method', 'mean_value', 'max_value', 'min_value', 'std_dev']

# Now, grouped_df contains the mean, max, min, and standard deviation of the 'value' column
# for each combination of 'embedding_method', 'metric', and 'state'.
grouped_df.head(200)


,metric,state,embedding_method,mean_value,max_value,min_value,std_dev
0,test_result_acc,AK,one_hot,0.732446,0.776932,0.387197,0.070916
1,test_result_acc,AK,table_to_text,0.732192,0.780598,0.463903,0.056516
2,test_result_acc,AL,one_hot,0.728420,0.768771,0.311209,0.076405
3,test_result_acc,AL,table_to_text,0.732028,0.765897,0.340129,0.046944
4,test_result_acc,AR,one_hot,0.723681,0.763228,0.268505,0.082000
...,...,...,...,...,...,...,...
195,test_result_f1,WI,table_to_text,0.692026,0.746304,0.309128,0.092608
196,test_result_f1,WV,one_hot,0.680046,0.731232,0.226358,0.096557
197,test_result_f1,WV,table_to_text,0.675277,0.726269,0.275293,0.082335
198,test_result_f1,WY,one_hot,0.688819,0.742260,0.264033,0.097343


In [21]:
# average performance difference between different methods

for col_name in ['mean_value', 'max_value', 'min_value']:
    for metric in metric_list:
        total_diff = 0
        for state in ACC_STATES:
            for embedding_method in embedding_method_list:
                table_to_text = grouped_df[(grouped_df['metric'] == metric) & (grouped_df['state'] == state) & (grouped_df['embedding_method'] == 'table_to_text')][col_name].values[0]
                one_hot = grouped_df[(grouped_df['metric'] == metric) & (grouped_df['state'] == state) & (grouped_df['embedding_method'] == 'one_hot')][col_name].values[0]
                diff = table_to_text - one_hot
                total_diff += diff
        average_diff = total_diff / len(ACC_STATES)
        print(f'Average {col_name} difference for {metric}: {average_diff * 100:.2f}%')


Average mean_value difference for test_result_acc: -0.45%
Average mean_value difference for test_result_f1: -2.09%
Average max_value difference for test_result_acc: -1.39%
Average max_value difference for test_result_f1: -1.91%
Average min_value difference for test_result_acc: 4.21%
Average min_value difference for test_result_f1: 11.62%
